#The Spiking Neuron Model - Coding Challenge Problems (Part 3)


#Hodgkin-Huxley Spiking Neuron Model

This interactive document is meant to be followed as the reader makes their way through chapter: *The Spiking Neuron Model*.  Each model presented in the chapter will have a section consisting of a step-by-step walkthrough of a simple Python implementation.  This is followed by an interface to run simulations with different parameter values to answer the Coding Challenge Problems.

For each model covered in the chapter, there is a section called **Simulations for Coding Challenge Problems.**  This is where you will find user-interface components such as value sliders for various parameters.  Use these controls to answer the questions from the text, and feel free to explore how the values affect the results.


**Content creators**: Maxwell E. Bohling, Dr. Lawrence Udeigwe

## How This Works

This Jupyter Notebook has both Content cells and Code cells.  As you make your way through the tutorial, you MUST make sure to run each code cell as you come to them.  Each code cell has a *Play* button next to it which will execute the code. Some code may be hidden (generally because it is more complex code not completely necessary to understand for the purposes of the chapter Coding Challenge Problems.)

**IMPORTANT**: You are currently viewing a copy of the original notebook.  You will find that you can edit the content of any cell.  If you accidently change a cell, such as a line of code and/or run into errors as you try to run subsequent blocks, simply refresh the page, OR go to the *Runtime* Menu and select *Restart runtime*.  It is also suggested that you go to the *Edit* menu and select *Clear all outputs.*  This will always allow you to revert the notebook to the original version (though you will have to run each code block again.)



 Execute the code block. **Initialize Setup**

In [ ]:
#@title Initialize Setup
#@markdown Import python libraries, and define helper functions (No need to understand this code, simply make sure you run this first.)
import sys
import functools as ft
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy as sc

def printError():
  message = 'Something went wrong!\n\n'  
  message = message + 'Check for the following:\n\n'
  message = message + '\t1.  Variables are set to the correct values.  Make sure to replace any instances of \'None\'.\n'
  message = message + '\t2.  All previous code blocks have been run the order they appear and output a success message.\n'
  message = message + '\t3.  No other code has been altered.\n\n'
  message = message + 'then try running the code block again.\n'
  print(message)

def printSuccess():
  message = 'Success!  Move on to the next section.'
  print(message)

def checkVoltageParameters(Vrest, Vth, Vreset, Vspike):
  # Check: IAF-Voltage-Parameters
  print('Checking Voltage Parameters...\n')
  try:
      check_Vrest = Vrest
      check_Vth = Vth
      check_Vreset = Vreset
      check_Vspike = Vspike
  except:
    printError()
  else:
    vals = [Vrest, Vth, Vreset, Vspike]
    correct_vals = [-70, -54, -80, 0]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  True):  
      printSuccess()
    else:
      printError()
 
def checkNeuronProperties(Rm, Cm, tau_m):
  # Check: IAF-Neuron-Properties
  print('Checking Neuron Properties...\n')
  try:
      check_Rm = Rm
      check_Cm = Cm
      check_tau_m = tau_m
  except:
    printError()
  else:
    vals = [Rm, Cm, tau_m]
    correct_vals = [10, 1, 10]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  True):  
      printSuccess()
    else:
      printError()

def checkSimulationSetup(Vrest, Vinitial, EL, t0, dt, t_final, time, Ie, t_pulse, start_current, end_current):
  # Check: IAF-Simulation-Setup
  print('Checking Simulation Setup...\n')
  try:
    check_Vrest = Vrest
    check_Vinitial = Vinitial
    check_EL = EL
    check_t0 = t0
    check_dt = dt
    check_t_final = t_final
    check_time = time
    check_Ie = Ie
    check_t_pulse = t_pulse
    check_start_current = start_current
    check_end_current = end_current
  except:
    printError()
  else:
    vals = [Vrest, Vinitial, EL, t0, dt, t_final, Ie, t_pulse, start_current, end_current]
    correct_vals = [-70, -70, -70, 0, 1, 500, 1.75, 300, 100, 400]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  True):  
      if len(time) == 500 and time[0] == 0 and time[-1] == 499:
        printSuccess()
      else:
        printError()
    else:
      printError()

def autoSetValues():
  Vrest = -70
  Vth = -54 
  Vreset = -80
  Vspike = 0
  Vinitial = Vrest
  EL = Vrest
  t0 = 0
  dt = 1
  t_final = 500
  time = range(t0, t_final, dt)
  t_pulse = 300 
  start_current = np.absolute(t_final-t_pulse)/2 
  end_current = start_current+t_pulse 
  V = [0] * len(time)
  V[0] = Vrest

try:
  check_sys = sys
except:
  printError()
else:
  modulename = 'functools'
  if modulename not in sys.modules:
    printError()
  else:
    printSuccess()

## Walkthrough
The goal of this section is to write a Python implementation of the Hodgkin-Huxley model.  Recall from the chapter text that we need to account for both activation and inactivation gating variables in order to simulate the persistent and transient conductances involved in the membrane current equation.

### Membrane Current
Recall that the Hodgkin-Huxley model is expressed as a sum of currents.  The membrane current equation is given as:

> $ \displaystyle i_{m} = \overline{g}_{L}(V-E_{L}) + \overline{g}_{K}n^4(V-E_{K}) + \overline{g}_{Na}m^3h(V-E_{Na})$

with maximal conductances $\overline{g}_{L},\;$ $\overline{g}_{K}\;$ $\overline{g}_{Na}\;$ and reversal potentials $E_{L},\;$ $E_{K},\;$ $E_{Na}$.

As with the previous models, Euler's method is used to compute the time evolution of the membrane potential $V$.  For this model, we use the same numerical integration method to compute the evolution of the gating variables $n$, $m$, and $h$. 

### Membrane Equation
Recall that the membrane equation is expressed as follows:

> $ \displaystyle \frac{dV}{dt} = -i_m+ \frac{I_{e}}{A} $

### Voltage Parameters

As opposed to the integrate-and-fire model, the Hodgkin-Huxley model does not utilize a spiking mechanism.  Therefore, we only need to define the *Voltage Parameter* that determine the resting potential:

*   $ V_{rest} = -70\;$*mV*




In [ ]:
try:
  # Voltage Parameters - Units mV (1 mV = 1e-3 Volts)
  Vrest = -70
except:
  printError()
else:
  printSuccess()

### Neuron Properties
The membrane equation is described by a total membrane current $i_{m}$ as a sum of:

1.  A *leakage current*: $ \displaystyle\; \overline{g}_{L}(V-E_{L}) $
2.  A *persistent current*:  $\displaystyle\; \overline{g}_{K}n^4(V-E_{K}) $
3.  A *transient current*:  $\displaystyle\; \overline{g}_{Na}m^3h(V-E_{Na})$

Thus, the persistent conductance is modeled as a K$^+$ conductance and the transient conductance is modeled as a Na$^+$ conductance.  For each current, we define the maximimal conductances:

*   $ \displaystyle\; \overline{g}_{L} = 0.3\;$mS / mm$^2$
*   $ \displaystyle\; \overline{g}_{K} = 35\;$mS / mm$^2$
*   $ \displaystyle\; \overline{g}_{Na} = 40\;$mS / mm$^2$

and reversal potentials:

*   $ \displaystyle\; E_{L} = -70\;$mV
*   $ \displaystyle\; E_{K} = -77\;$mV
*   $ \displaystyle\; E_{Na} = 55\;$mV

Lastly, as seen in the membrane equation for the model, we must define the value of the injected current, and the neuronal surface area:

*   $ \displaystyle\; I_{e} = 1\;$nA
*   $ \displaystyle\; A = 0.1\;$mm$^2$

In [ ]:
try:
  #Maximal Conductances - Units mS/mm^2
  GL = 0.3
  GK = 35
  GNa = 40

  # Reversal Potentials - Units mV
  EL = -70;
  EK = -77
  ENa = 55

  # Input current: Ie - Units nA (1 nA = 10-9 Amperes)
  Ie = 1.00

  # Neuron Surface Area - Units mm^2
  A = 0.1
except:
  printError()
else:
  printSuccess()

### Simulation Setup
We will be running a 20 ms simulation.  The following lines of code setup a time span for the simulation.  This is simply a matter of defining the start time $t_{0} = 0$ and the total length (in ms) of the simulation: $t_{final} = 20$.  

Throughout the simulation, we calculate the membrane potential $V$ at each *time-step*.  The time-step is the change in time for each iteration of the simulation, for example if $t_{0} = 0$, the next computation of $V$ is performed at $t_{0} + dt$.  

Thus, by setting $dt = 0.01$ (in ms), the simulation will compute $V$, $n$, $m$, and $h$ at time $t = 1, 2, \ldots, t_{final}$. 

In [ ]:
try:
  # Simulation Time Span (0 to 20ms, dt = 0.01ms)
  t0 = 0
  dt = 0.01
  t_final = 20

  time = np.linspace(t0, t_final,  2000)
except:
  printError()
else:
  if len(time) == 2000 and time[0] == 0 and time[-1] == 20:
    printSuccess()
  else:
    printError()

Next, we define the time $t$ at which the injected current $I_{e}$ is *switched on* and applied to the neuron, and the time $t$ at which the injected current is *switched off*.

For the Hodgkin-Huxley model, we run a shorter simulation and we apply the current from $t = 1\;$ms to $t = 2\;$ms.

In [ ]:
try:
  # Time at which the current is applied - Units ms
  start_current = 1
  
  # Time at which the current is switched off - Units ms
  end_current = 2
except:
  printError()
else:
  printSuccess()

To setup our simulation, we need initial values of each variable: $V$, $n$, $m$, and $h$ as well as a list to hold the values over time.

Set initial values as:

* $V_{initial}= V_{rest}\;$*mV*
* $n_{initial} = 0.1399$
* $m_{initial} = 0.0498$
* $h_{initial} = 0.6225$

With each value defined at time $t = 0$, let $V_0 = V_{initial}, n_0 = n_{initial},  m_0 = m_{initial}, h_0 = h_{initial} $.  

The initial membrane current is then:

*  $\displaystyle i_{initial} = \overline{g}_{L}(V_0-E_{L}) + \overline{g}_{K}n_0^4(V_0-E_{K}) + \overline{g}_{Na}m_0^3h_0(V_0-E_{Na})$ 

In [ ]:
try:
  # Initial voltage
  Vinitial = Vrest

  # Create a list V(t) to store the value of V at each time-step dt
  V = [0] * len(time)

  # Set the initial value at time t = t0 to the initial value Vinitial
  V[0] =  Vinitial

  # Initial gating variable values (Probability [0, 1])
  n_initial = 0.1399
  m_initial = 0.0498
  h_initial = 0.6225

  # Create lists to store the value of each gating variable at each time-step dt
  n = [0] * len(time)
  m= [0] * len(time)
  h = [0] * len(time)

  # Set the initial value at time t = t0 to the initial values
  n[0] =  n_initial
  m[0] =  m_initial
  h[0] =  h_initial

  # Initial membrane current 
  im_initial = GL*(V[0]-EL)+GK*np.power(n[0],4)*(V[0]-EK)+GNa*np.power(m[0],3)*h[0]*(V[0]-ENa)

  # Create list to store value of membrane current at each time-step dt
  im = [0] * len(time)

  # Set the initial value at time t = t0 to the initial value im_initial
  im[0] = im_initial
except:
  printError()
else:
  printSuccess()

### Computing and Storing $\frac{dV}{dt}$, $\frac{dn}{dt}$, $\frac{dm}{dt}$, $\frac{dh}{dt}$

### Opening and Closing Rate Functions for Gating Variables

The gating variables $n$, $m$, and $h$ represent **probabilities** that a gate mechanism in both the persistent and transient ion-conducting channels are open or *activated*.   

For any arbitrary gating variable $z$,  the open probability of a channel at any time $ t $ is computed using  an *opening* rate function $\alpha_{z}(V)$ and a *closing* rate $\beta_{z}(V)$, both of which are functions of the membrane potential $V$.

Each gating variable is numerically integrated using Euler's method throughout the simulation, where for any arbitrary gating variable $z$, the rate functions are given as follows:

> $ \displaystyle \tau_{z}(V)\frac{dz}{dt} = z_{\infty}(V) - z $

where

>  $ \displaystyle \tau_{z}(V) = \frac{1}{\alpha_{z}(V) + \beta_{z}(V)} $

and

> $ \displaystyle z_{\infty}(V) = \frac{\alpha_{z}(V) }{\alpha_{z}(V) + \beta_{z}(V)} $




#### Fitted Rate Functions
Hodgkin and Huxley had fit the opening and closing rate functions using experimental data.  These are given as follows:

For activation variable $n$

> $ \displaystyle \alpha_{n}(V) = \frac{0.01(V+60)}{ 1 - \exp(V+60)} $

> $ \displaystyle \beta_{n}(V) = 0.125\exp(-0.0125(V+70)) $

For activation variable $m$

> $ \displaystyle \alpha_{m}(V) = \frac{0.1(V+45)}{1 - \exp(-0.1(V+45))}$

> $ \displaystyle \beta_{m}(V) = 4\exp(-0.0556(V+70)) $

For inactivation variable $h$

> $ \displaystyle \alpha_{h}(V) = .07\exp(-0.05(V+70))$

> $ \displaystyle \beta_{h}(V) = \frac{1}{1 + \exp(-0.1(V+40))} $

We define separate functions for each gating variable.  These will take the membrane potential $V(t)$ as input, and ouput $dz$ where $z = n, m, h $.  Using the functional forms and fitted rate functions, these functions compute the changes dn, dm, and dh at each time-step dt which depend on the membrane potential V at time t.


In [ ]:
# Function: compute_dn
def compute_dn(v, n):
  alpha_n = (0.01*(v + 60))/(1 - np.exp(v+60))
  beta_n = 0.125*np.exp(-0.0125*(v+70))

  n_inf = alpha_n/(alpha_n + beta_n)
  tau_n = 1/(alpha_n + beta_n)

  dn = (dt/tau_n)*(n_inf - n)
  return dn

# Function: compute_dm
def compute_dm(v, m):
  alpha_m = (0.1*(v + 45))/(1 - np.exp(-0.1*(v+45)))
  beta_m = 4*np.exp(-0.0556*(v+70))

  m_inf = alpha_m/(alpha_m + beta_m)
  tau_m = 1/(alpha_m + beta_m)

  dm = (dt/tau_m)*(m_inf - m)
  return dm

# Function: compute_dh
def compute_dh(v, h):
  alpha_h = 0.07*np.exp(-0.05*(v+70))
  beta_h =  1/(1 + np.exp(-0.1*(v+40)))

  h_inf = alpha_h/(alpha_h + beta_h)
  tau_h = 1/(alpha_h + beta_h)

  dh = (dt/tau_h)*(h_inf - h)
  return dh

Finally, we run our simulation according to the updated *pseudocode*

---

*for each time-step from $t = t_{0}$ to $t = t_{final}$*
> *If the current time $t \geq start_{current}\ $ and $\ t \leq end_{current}$*
>> $I_{e} = 1\;$nA

> *otherwise*
>> $I_{e} = 0\;$nA

> *First compute the open probabilites for each gating variable*

> $ \displaystyle dn = $ **compute_dn**$(V(t), n(t))$

> *Update* $ n(t+1) = n(t) + dn $

> $ \displaystyle dm = $ **compute_dm**$(V(t), m(t))$

> *Update* $ m(t+1) = m(t) + dm $

> $ \displaystyle dh = $ **compute_dh**$(V(t), h(t))$

> *Update* $ h(t+1) = h(t) + dh $

> $ \displaystyle i_{m}(t+1) = \overline{g}_{L}(V(t)-E_{L}) + \overline{g}_{K}n(t+1)^4(V(t)-E_{K}) + \overline{g}_{Na}m(t+1)^3h(t+1)(V(t)-E_{Na})$

> *Use Euler's Method of Numerical Integration*

> $ \displaystyle dV= dt\left(-i_m(t+1)+ \frac{I_{e}}{A}\right) $

> *Update* $V(t+1) = V(t) + dV$


*end*

---

This translates to the following Python code.

In [ ]:
try:
  # For each timestep we compute V and store the value
  for t in range(len(time)-2):

    # If time t >= 1 ms and t <= 2 ms, switch Injected Current ON
    if time[t] >= start_current and time[t]  <= end_current:
      ie = Ie
    # Otherwise, switch Injected Current OFF
    else:
      ie = 0

    # For each timestep we compute n, m and h and store the value
    dn = compute_dn(V[t], n[t])
    n[t+1] = n[t] + dn

    dm = compute_dm(V[t], m[t])
    m[t+1] = m[t] + dm

    dh = compute_dh(V[t], h[t])
    h[t+1] = h[t] + dh

    # Use these values to compute the updated membrane current
    im[t+1] = GL*(V[t]-EL)+GK*np.power(n[t+1],4)*(V[t]-EK)+GNa*np.power(m[t+1],3)*h[t+1]*(V[t]-ENa)

    # Using Euler's Method for Numerical Integration (See Chapter Text)
    # we compute the change in voltage dV as follows (using the model equation)
    dV = dt*(-1*im[t+1] + ie/A)

    # Store this new value into our list
    V[t+1] = V[t] + dV
except:
  printError()
else:
  printSuccess()

### Visualizing Results

Now we have values of $V$, $i_m$, $n$, $m$, and $h$ for each time-step of the simulation, we can visualize the results by using Python to plot the data.  This makes use of another widely used library **matplotlib**.

In [ ]:
try:
  check_plt = plt
except:
  printError()
else:
  fig, axs = plt.subplots(nrows=3, ncols=1, sharex= True)

  axs[0].plot(time[0:-2], V[0:-2])
  axs[0].set_title('Hodgkin-Huxley: $V$ against Time')
  axs[0].set_xlabel('Time (ms)')
  axs[0].set_ylabel('$V(t)$')

  axs[1].plot(time[0:-2], im[0:-2])
  axs[1].set_title('$i_{m}$ against Time')
  axs[1].set_xlabel('Time (ms)')
  axs[1].set_ylabel('$i_{m}\;$mV')

  axs[2].plot(time[0:-2], n[0:-2])#, m[0:-2]))
  axs[2].plot(time[0:-2], m[0:-2])
  axs[2].plot(time[0:-2], h[0:-2])
  axs[2].set_title('$n, h, m$ against Time')
  axs[2].set_xlabel('Time (ms)')
  axs[2].set_ylabel('$n, h, m$')

  # Display the plot
  plt.show()    

## Hodgkin-Huxley Spiking Neuron Model - Full Code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Voltage Parameters - Units mV (1 mV = 1e-3 Volts)
Vrest = -70

#Maximal Conductances - Units mS/mm^2
GL = 0.3
GK = 35
GNa = 40

# Reversal Potentials - Units mV
EL = -70;
EK = -77
ENa = 55

# Input current: Ie - Units nA (1 nA = 10-9 Amperes)
Ie = 1.00

# Neuron Surface Area - Units mm^2
A = 0.1

# Simulation Time Span (0 to 20ms, dt = 0.01ms)
t0 = 0
dt = 0.01
t_final = 20
time = np.linspace(t0, t_final,  2000)

# Time at which the current is applied - Units ms
start_current = 1

# Time at which the current is switched off - Units ms
end_current = 2

# Initial voltage
Vinitial = Vrest

# Create a list V(t) to store the value of V at each time-step dt
V = [0] * len(time)

# Set the initial value at time t = t0 to the initial value Vinitial
V[0] =  Vinitial

# Initial gating variable values (Probability [0, 1])
n_initial = 0.1399
m_initial = 0.0498
h_initial = 0.6225

# Create lists to store the value of each gating variable at each time-step dt
n = [0] * len(time)
m= [0] * len(time)
h = [0] * len(time)

# Set the initial value at time t = t0 to the initial values
n[0] =  n_initial
m[0] =  m_initial
h[0] =  h_initial

# Initial membrane current 
im_initial = GL*(V[0]-EL)+GK*np.power(n[0],4)*(V[0]-EK)+GNa*np.power(m[0],3)*h[0]*(V[0]-ENa)

# Create list to store value of membrane current at each time-step dt
im = [0] * len(time)

# Set the initial value at time t = t0 to the initial value im_initial
im[0] = im_initial

# For each timestep we compute V and store the value
for t in range(len(time)-2):

  # If time t >= 1 ms and t <= 2 ms, switch Injected Current ON
  if time[t] >= start_current and time[t]  <= end_current:
    ie = Ie
  # Otherwise, switch Injected Current OFF
  else:
    ie = 0

  # For each timestep we compute n, m and h and store the value
  dn = compute_dn(V[t], n[t])
  n[t+1] = n[t] + dn

  dm = compute_dm(V[t], m[t])
  m[t+1] = m[t] + dm

  dh = compute_dh(V[t], h[t])
  h[t+1] = h[t] + dh

  # Use these values to compute the updated membrane current
  im[t+1] = GL*(V[t]-EL)+GK*np.power(n[t+1],4)*(V[t]-EK)+GNa*np.power(m[t+1],3)*h[t+1]*(V[t]-ENa)

  # Using Euler's Method for Numerical Integration (See Chapter Text)
  # we compute the change in voltage dV as follows (using the model equation)
  dV = dt*(-1*im[t+1] + ie/A)

  # Store this new value into our list
  V[t+1] = V[t] + dV

fig, axs = plt.subplots(nrows=3, ncols=1, sharex= True)

axs[0].plot(time[0:-2], V[0:-2])
axs[0].set_title('Hodgkin-Huxley: $V$ against Time')
axs[0].set_xlabel('Time (ms)')
axs[0].set_ylabel('$V(t)$')

axs[1].plot(time[0:-2], im[0:-2])
axs[1].set_title('$i_{m}$ against Time')
axs[1].set_xlabel('Time (ms)')
axs[1].set_ylabel('$i_{m}\;$mV')

axs[2].plot(time[0:-2], n[0:-2])#, m[0:-2]))
axs[2].plot(time[0:-2], m[0:-2])
axs[2].plot(time[0:-2], h[0:-2])
axs[2].set_title('$n, h, m$ against Time')
axs[2].set_xlabel('Time (ms)')
axs[2].set_ylabel('$n, h, m$')

# Display the plot
plt.show() 

# Function: compute_dn
def compute_dn(v, n):
  alpha_n = (0.01*(v + 60))/(1 - np.exp(v+60))
  beta_n = 0.125*np.exp(-0.0125*(v+70))

  n_inf = alpha_n/(alpha_n + beta_n)
  tau_n = 1/(alpha_n + beta_n)

  dn = (dt/tau_n)*(n_inf - n)
  return dn

# Function: compute_dm
def compute_dm(v, m):
  alpha_m = (0.1*(v + 45))/(1 - np.exp(-0.1*(v+45)))
  beta_m = 4*np.exp(-0.0556*(v+70))

  m_inf = alpha_m/(alpha_m + beta_m)
  tau_m = 1/(alpha_m + beta_m)

  dm = (dt/tau_m)*(m_inf - m)
  return dm

# Function: compute_dh
def compute_dh(v, h):
  alpha_h = 0.07*np.exp(-0.05*(v+70))
  beta_h =  1/(1 + np.exp(-0.1*(v+40)))

  h_inf = alpha_h/(alpha_h + beta_h)
  tau_h = 1/(alpha_h + beta_h)

  dh = (dt/tau_h)*(h_inf - h)
  return dh

## Simulations for Coding Challenge Problems